# SageMaker Clarify E2E Test

Simple end-to-end test for the Clarify utils implementation

In [2]:
import sys
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib
import boto3

# Add the clarify utils to path
# sys.path.insert(0, '/Users/mollyhe/Documents/SageMaker/sagemaker-python-sdk-staging-molly/sagemaker_utils/src')

from sagemaker.core.clarify import (
    SageMakerClarifyProcessor,
    DataConfig,
    BiasConfig,
    ModelConfig,
    SHAPConfig
)
from sagemaker.core.helper.session_helper import Session,get_execution_role
role = get_execution_role()

[11/08/25 18:39:19] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=479005;file:///Users/mollyhe/.pyenv/versions/3.12.2/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=274417;file:///Users/mollyhe/.pyenv/versions/3.12.2/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

## 1. Create Sample Data

In [3]:
# Create synthetic dataset
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=5,
    n_redundant=2,
    random_state=42
)

# Add a sensitive feature (simulating gender: 0=female, 1=male)
sensitive_feature = np.random.binomial(1, 0.4, size=X.shape[0])
X = np.column_stack([X, sensitive_feature])

# Create DataFrame
feature_names = [f'feature_{i}' for i in range(10)] + ['gender']
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

print(f"Dataset shape: {df.shape}")
print(f"Target distribution: {df['target'].value_counts()}")
print(f"Gender distribution: {df['gender'].value_counts()}")

Dataset shape: (1000, 12)
Target distribution: target
1    503
0    497
Name: count, dtype: int64
Gender distribution: gender
0.0    598
1.0    402
Name: count, dtype: int64


## 2. Train Simple Model

In [4]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42
)

# Train model
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

print(f"Model accuracy: {model.score(X_test, y_test):.3f}")

Model accuracy: 0.920


## 3. Upload Data to S3

In [5]:
# Setup S3 paths
session = Session()
bucket = session.default_bucket()
prefix = 'clarify-test'

# Save test data (without target for inference)
test_data = X_test.copy()
test_data['target'] = y_test
test_data.to_csv('/tmp/test_data.csv', index=False)

# Save model
joblib.dump(model, '/tmp/model.joblib')

# Upload to S3
s3_client = boto3.client('s3')
s3_client.upload_file('/tmp/test_data.csv', bucket, f'{prefix}/data/test_data.csv')
s3_client.upload_file('/tmp/model.joblib', bucket, f'{prefix}/model/model.joblib')

data_uri = f's3://{bucket}/{prefix}/data/test_data.csv'
output_uri = f's3://{bucket}/{prefix}/output'

print(f"Data uploaded to: {data_uri}")
print(f"Output will be saved to: {output_uri}")

[11/08/25 18:39:30] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=514072;file:///Users/mollyhe/.pyenv/versions/3.12.2/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=337498;file:///Users/mollyhe/.pyenv/versions/3.12.2/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

[11/08/25 18:39:31] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=820880;file:///Users/mollyhe/.pyenv/versions/3.12.2/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=366626;file:///Users/mollyhe/.pyenv/versions/3.12.2/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

Data uploaded to: s3://sagemaker-us-west-2-529088288990/clarify-test/data/test_data.csv
Output will be saved to: s3://sagemaker-us-west-2-529088288990/clarify-test/output


## 4. Configure Clarify

In [6]:
# Data configuration
data_config = DataConfig(
    s3_data_input_path=data_uri,
    s3_output_path=output_uri,
    label='target',
    headers=list(test_data.columns),
    dataset_type='text/csv'
)

# Bias configuration
bias_config = BiasConfig(
    label_values_or_threshold=[1],  # Positive class
    facet_name='gender',
    facet_values_or_threshold=[1]   # Male as sensitive group
)

# SHAP configuration
shap_config = SHAPConfig(
    baseline=None,  # Auto-generate baseline
    num_samples=10,  # Small number for quick test
    agg_method='mean_abs'
)

print("Configurations created successfully")

Configurations created successfully


## 5. Create Clarify Processor

In [7]:
# Create Clarify processor
clarify_processor = SageMakerClarifyProcessor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    sagemaker_session=session
)

print(f"Clarify processor created with role: {role}")

[11/08/25 18:39:35] INFO     Ignoring unnecessary instance type: None.                            ]8;id=687743;file:///Users/mollyhe/.pyenv/versions/3.12.2/lib/python3.12/site-packages/sagemaker/utils/image_uris.py\image_uris.py]8;;\:]8;id=190982;file:///Users/mollyhe/.pyenv/versions/3.12.2/lib/python3.12/site-packages/sagemaker/utils/image_uris.py#529\529]8;;\

Clarify processor created with role: arn:aws:iam::529088288990:role/Admin


## 6. Run Pre-training Bias Analysis

In [8]:
# Run pre-training bias analysis (no model needed)
try:
    clarify_processor.run_pre_training_bias(
        data_config=data_config,
        data_bias_config=bias_config,
        methods=['CI', 'DPL'],  # Class Imbalance and Difference in Positive Proportions
        wait=False,  # Don't wait for completion in test
        logs=False
    )
    print("✅ Pre-training bias analysis job submitted successfully")
except Exception as e:
    print(f"❌ Pre-training bias analysis failed: {str(e)}")

[11/08/25 18:39:36] INFO     Analysis Config: {'dataset_type': 'text/csv', 'headers': ['feature_0', ]8;id=70589;file:///Users/mollyhe/.pyenv/versions/3.12.2/lib/python3.12/site-packages/sagemaker/utils/clarify.py\clarify.py]8;;\:]8;id=487060;file:///Users/mollyhe/.pyenv/versions/3.12.2/lib/python3.12/site-packages/sagemaker/utils/clarify.py#1992\1992]8;;\
                             'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',                      
                             'feature_6', 'feature_7', 'feature_8', 'feature_9', 'gender',                         
                             'target'], 'label': 'target', 'label_values_or_threshold': [1],                       
                             'facet': [{'name_or_index': 'gender', 'value_or_threshold': [1]}],                    
                             'methods': {'report': {'name': 'report', 'title': 'Analysis Report'},                 
                             'pre_training_bias': {'methods': ['CI', 'DPL']}}}                                     

                    INFO     Creating processing-job with name                                    ]8;id=100415;file:///Users/mollyhe/.pyenv/versions/3.12.2/lib/python3.12/site-packages/sagemaker/utils/processing.py\processing.py]8;;\:]8;id=219709;file:///Users/mollyhe/.pyenv/versions/3.12.2/lib/python3.12/site-packages/sagemaker/utils/processing.py#598\598]8;;\
                             Clarify-Pretraining-Bias-2025-11-09-02-39-36-699                                      

✅ Pre-training bias analysis job submitted successfully


In [ ]:
# You can go to SageMaker AI console -> Processing jobs and check the job status
# Or you can run the below command
# Note that it takes ~5min for the job to be complete

response = session.sagemaker_client.describe_processing_job(ProcessingJobName='Clarify-Pretraining-Bias-2025-11-09-02-39-36-699')
print(f"Status: {response['ProcessingJobStatus']}")

Status: Completed


## 7. Test Configuration Generation

In [14]:
# Test the internal config generation
from sagemaker.core.clarify import _AnalysisConfigGenerator

try:
    # Generate bias config
    bias_analysis_config = _AnalysisConfigGenerator.bias_pre_training(
        data_config=data_config,
        bias_config=bias_config,
        methods=['CI', 'DPL']
    )
    
    print("✅ Bias analysis config generated successfully")
    print(f"Config keys: {list(bias_analysis_config.keys())}")
    
    # Validate config structure
    required_keys = ['dataset_type', 'label_values_or_threshold', 'facet', 'methods']
    missing_keys = [key for key in required_keys if key not in bias_analysis_config]
    
    if missing_keys:
        print(f"❌ Missing required keys: {missing_keys}")
    else:
        print("✅ All required keys present in config")
        
except Exception as e:
    print(f"❌ Config generation failed: {str(e)}")

✅ Bias analysis config generated successfully
Config keys: ['dataset_type', 'headers', 'label', 'label_values_or_threshold', 'facet', 'methods']
✅ All required keys present in config


## 8. Test Schema Validation

In [15]:
# Test schema validation
from sagemaker.core.clarify import ANALYSIS_CONFIG_SCHEMA_V1_0

try:
    # Validate the generated config
    ANALYSIS_CONFIG_SCHEMA_V1_0.validate(bias_analysis_config)
    print("✅ Schema validation passed")
except Exception as e:
    print(f"❌ Schema validation failed: {str(e)}")

✅ Schema validation passed
